In [1]:
# Vizro-AI usage

from vizro import Vizro
import vizro_ai
from dotenv import load_dotenv

print(vizro_ai.__version__)
load_dotenv()

from vizro_ai import VizroAI
vizro_ai = VizroAI(model="gpt-4o")

import pandas as pd

df = pd.read_csv('filtered_books.csv')
df["Date Read"] = pd.to_datetime(df['Date Read'],dayfirst=True)

# Data cleanup 
# Specify columns to check for missing values
columns_to_check = ['Title', 'Author', 'ISBN', 'ISBN13', 'My Rating', 'Average Rating', 'Number of Pages', 'Original Publication Year', 'Date Read']
df_cleaned = df.dropna(subset=columns_to_check)
#df_cleaned.head()

0.3.2


In [2]:
another_user_question = """
Create a dashboard with 3 pages. 

On the first page, plot a chart with the title "Sequence of reading" . It is a scatter chart. 
Use the x axis to show the date a book was read. Plot it at y=1.

On the second page, plot a chart with the title "Pages and Book totals" . It shows the cumulative total number of pages read by summing the Number of Pages of each book read in each year, using the Date Read data. Plot date on the x axis and the number of pages on the y axis using a scale on the left hand side of the chart.

Add a filter so the user can change the x axis to adjust the range of dates by year on the x axis.

Superimpose a bar chart showing the total books read for each year, taking data from the Date Read column. 

Show the total books read using the right hand side of the chart which can be a different scale to the y axis shown on the left hand side.

On the third page, plot a chart with the title "Ratings over the years".

Eliminate any book where My Rating is 0. Then, for each row of the dataset, plot the My Rating and Average Rating 

data points using circles that are connected by a line to show the gap between the two points. 

Stack the books vertically on the y axis, ordered alphabetically by Title, and use the x axis to show the rating between 0 and 5. Label each with the Title and Author.


"""

In [3]:
result = vizro_ai.dashboard([df_cleaned], another_user_question, return_elements=True)
Vizro().build(result.dashboard).run(port=8006)
print(result.code)

Store df info:   0%|          | 0/1 [00:00<?, ?it/s]

df_name: book_reading_data


Generate dashboard plan:   0%|          | 0/2 [00:00<?, ?it/s]

Building page: Sequence of Reading:   0%|          | 0/5 [00:00<?, ?it/s]

Building page: Pages and Book Totals:   0%|          | 0/5 [00:00<?, ?it/s]

...

Currently Building ... [Page] <Sequence of Reading> controls: 0it [00:00, ?it/s]

Currently Building ... [Page] <Pages and Book Totals> controls:   0%|          | 0/1 [00:00<?, ?it/s]

/Users/jo_stichbury/Documents/GitHub/vizro/vizro-core/vizro/lib/python3.10/site-packages/pydantic/v1/schema.py:631: UserWarning:

CapturedCallable function is excluded from the schema.

[FALLBACK] Failed to build `Component`: ratings_over_the_years.
Reason: 1 validation error for ChartPlanDynamic
chart_code
  Produced code failed the safeguard validation: <Unsafe builtin functions reversed are used in generated code line: fig.update_layout(title='Ratings over the years', xaxis_title='Rating', yaxis_title='Books', xaxis=dict(range=[0, 5]), yaxis=dict(autorange='reversed'), showlegend=False) and cannot be executed. If you require a builtin package, reach out to the Vizro team.>. Please check the code and try again. (type=value_error)
Relevant prompt: Chart titled 'Ratings over the years'. Eliminates books where My Rating is 0. For each row, plots My Rating and Average Rating data points using circles connected by a line to show the gap. Books stacked vertically on y-axis, ordered alphabe

Currently Building ... [Page] <Ratings Over the Years> controls: 0it [00:00, ?it/s]

############ Imports ##############
import vizro.models as vm
from vizro.models.types import capture
import pandas as pd
import plotly.graph_objects as go
from vizro.models.types import capture


####### Function definitions ######
@capture("graph")
def pages_and_book_totals(data_frame):
    # Convert 'Date Read' to datetime
    data_frame["Date Read"] = pd.to_datetime(data_frame["Date Read"])

    # Extract year from 'Date Read'
    data_frame["Year Read"] = data_frame["Date Read"].dt.year

    # Group by 'Year Read' and calculate cumulative sum of 'Number of Pages'
    pages_per_year = data_frame.groupby("Year Read")["Number of Pages"].sum().cumsum()

    # Count total books read per year
    books_per_year = data_frame.groupby("Year Read").size()

    # Create figure
    fig = go.Figure()

    # Add cumulative pages line
    fig.add_trace(
        go.Scatter(
            x=pages_per_year.index,
            y=pages_per_year,
            mode="lines",
            name="Cumulative Page

In [ ]:

############ Imports ##############
from vizro import Vizro
import vizro.plotly.express as px
import vizro.tables as vt
import vizro.models as vm
from vizro.models.types import capture
import pandas as pd
import vizro.plotly.express as px
import plotly.graph_objects as go
from vizro.models.types import capture

import pandas as pd
import plotly.graph_objects as go
import vizro.plotly.express as px
from vizro.models.types import capture


####### Function definitions ######
@capture("graph")
def scatter_chart(data_frame):
    # Filter the data where My Rating is greater than 0
    filtered_data = data_frame[data_frame["My Rating"] > 0]

    # Create a scatter plot
    fig = px.scatter(
        filtered_data,
        x="Date Read",
        y="My Rating",
        hover_data={"Title": True, "Author": True},
        labels={"Date Read": "Date Read", "My Rating": "My Rating"},
        title="Books Read with My Rating",
    )

    return fig


@capture("graph")
def custom_chart(data_frame):
    fig = go.Figure()
    # Add My Rating points
    fig.add_trace(
        go.Scatter(
            x=data_frame["My Rating"],
            y=data_frame["Title"],
            mode="markers",
            name="My Rating",
            marker=dict(color="blue", size=10),
        )
    )
    # Add Average Rating points
    fig.add_trace(
        go.Scatter(
            x=data_frame["Average Rating"],
            y=data_frame["Title"],
            mode="markers",
            name="Average Rating",
            marker=dict(color="orange", size=10),
        )
    )
    # Add lines connecting the ratings
    for index, row in data_frame.iterrows():
        fig.add_trace(
            go.Scatter(
                x=[row["My Rating"], row["Average Rating"]],
                y=[row["Title"], row["Title"]],
                mode="markers+lines",
                line=dict(color="gray", width=2),
                marker=dict(color=["blue", "red"], size=10),
                showlegend=False,
            )
        )
    fig.update_layout(
        title="Dumbbell Chart: My Rating vs Average Rating",
        xaxis_title="Rating",
        yaxis_title="Books",
        xaxis=dict(range=[0, 5]),
        yaxis=dict(tickmode="linear"),
        height=600,
    )
    
    return fig

@capture("graph")
def pages_books_totals_chart(data_frame):
    # Convert 'Date Read' to datetime
    data_frame["Date Read"] = pd.to_datetime(data_frame["Date Read"])

    # Sort data by 'Date Read'
    data_frame = data_frame.sort_values("Date Read")

    # Calculate cumulative sum of 'Number of Pages'
    data_frame["Cumulative Pages"] = data_frame["Number of Pages"].cumsum()

    # Extract year from 'Date Read'
    data_frame["Year"] = data_frame["Date Read"].dt.year

    # Count total books read per year
    books_per_year = data_frame.groupby("Year").size()

    # Create figure
    fig = go.Figure()

    # Add line chart for cumulative pages
    fig.add_trace(
        go.Scatter(
            x=data_frame["Date Read"],
            y=data_frame["Cumulative Pages"],
            mode="lines",
            name="Cumulative Pages",
        )
    )

    # Add bar chart for total books read per year
    fig.add_trace(
        go.Bar(
            x=books_per_year.index,
            y=books_per_year.values,
            name="Total Books Read",
            yaxis="y2",
            opacity=0.5,
        )
    )

    # Update layout
    fig.update_layout(
        title="Cumulative Pages Read and Total Books Read Per Year",
        xaxis_title="Date",
        yaxis_title="Cumulative Pages",
        yaxis2=dict(title="Total Books Read", overlaying="y", side="right"),
        legend=dict(x=0.1, y=0.9),
    )

    return fig

df = pd.read_csv('filtered_books.csv')
df["Date Read"] = pd.to_datetime(df['Date Read'],dayfirst=True)


########### Model code ############
model = vm.Dashboard(
    pages=[
        vm.Page(
            components=[
                vm.AgGrid(
                    id="recent_reading_grid",
                    figure=vt.dash_ag_grid(df),
                )
            ],
            title="Recent reading",
            layout=vm.Layout(grid=[[0]]),
            controls=[],
        ),
        vm.Page(
            components=[
                vm.Graph(
                    id="pages_books_totals_chart",
                    figure=pages_books_totals_chart(df),
                )
            ],
            title="Pages and Book totals",
            layout=vm.Layout(grid=[[0]]),
            controls=[
                vm.Filter(
                    column="Year Published",
                    targets=["pages_books_totals_chart"],
                    selector=vm.RangeSlider(type="range_slider", title="Year Range"),
                )
            ],
        ),
        vm.Page(
            components=[
                vm.Graph(
                    id="custom_chart",
                    figure=custom_chart(df),
                ),
                vm.Graph(
                    id="scatter_chart",
                    figure=scatter_chart(df),
                ),
            ],
            title="Types of book",
            layout=vm.Layout(grid=[[0, 1]]),
            controls=[],
        ),
    ],
    title="Book Reading Dashboard",
)

Vizro().build(model).run()



##########################################################################################
##########################################################################################
##########################################################################################
##########################################################################################
##########################################################################################
##########################################################################################
"""
import plotly.graph_objects as go
from vizro.models.types import capture
@capture("graph")
def custom_chart(data_frame):
    fig = go.Figure()
    # Add My Rating points
    fig.add_trace(
        go.Scatter(
            x=data_frame["My Rating"],
            y=data_frame["Title"],
            mode="markers",
            name="My Rating",
            marker=dict(color="blue", size=10),
        )
    )
    # Add Average Rating points
    fig.add_trace(
        go.Scatter(
            x=data_frame["Average Rating"],
            y=data_frame["Title"],
            mode="markers",
            name="Average Rating",
            marker=dict(color="orange", size=10),
        )
    )
    # Add lines connecting the ratings
    for index, row in data_frame.iterrows():
        fig.add_trace(
            go.Scatter(
                x=[row["My Rating"], row["Average Rating"]],
                y=[row["Title"], row["Title"]],
                mode="markers+lines",
                line=dict(color="gray", width=2),
                marker=dict(color=["blue", "red"], size=10),
                showlegend=False,
            )
        )
    fig.update_layout(
        title="Dumbbell Chart: My Rating vs Average Rating",
        xaxis_title="Rating",
        yaxis_title="Books",
        xaxis=dict(range=[0, 5]),
        yaxis=dict(tickmode="linear"),
        height=600,
    )
    return fig
    """

In [ ]:
## DO NOT RUN THIS ONE

user_question = """
Create a dashboard page with title "Recent reading" showing 4 charts.

The first chart shows the cumulative total number of pages read by summing the Number of Pages of each book read in each year, using the Date Read data.
Plot the year on the x axis and the number of pages on the y axis using a scale on the left hand side of the chart. Superimpose a bar chart showing the total books read for each year, taking data from the Date Read column. 
Show the total books read using the right hand side of the chart which can be a different scale to the y axis shown on the left hand side.

The second is a bar chart where x is the Original Publication Year is ascending order. For the dataset, calculate the total number of books for each Original Publicatiokn year and plot this as y.

The third chart is a tornado chart which shows takes each row of the dataset and plots My Rating on the left and Average Rating on the right. Label each bar with the Title. 

The fourth chart is a pie chart which shows the number of books for each of the Categories. Do not include the N/A rows. Combine ['Fiction'] and ['FICTION'] and remove the [' and '] from each label.

"""

In [ ]:
# Following cells are all manipulation of the data

In [ ]:
import requests
import csv

def get_book_info_by_isbn(isbn):
    # Define the endpoint and the parameters
    endpoint = "https://www.googleapis.com/books/v1/volumes"
    params = {
        'q': 'isbn:' + isbn
    }

    # Make the GET request to the Google Books API
    response = requests.get(endpoint, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        book_data = response.json()
        if 'items' in book_data:
            # Return the first book found
            return book_data['items'][0]
        else:
            return None
    else:
        # Handle errors
        print(f"Error: {response.status_code}")
        return None


# File paths for input and output CSVs
input_csv_path = 'goodreads_library_export.csv'
output_csv_path = 'another_updated_goodreads_library_export.csv'

# Read the input CSV and process each row to add Categories and Small Thumbnail columns
with open(input_csv_path, mode='r', newline='') as infile, open(output_csv_path, mode='w', newline='') as outfile:
    reader = csv.DictReader(infile)

    # Add new fields "Categories" and "SmallThumbnail" to the existing fieldnames
    fieldnames = reader.fieldnames + ['Categories', 'SmallThumbnail']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    # Write the header to the output CSV
    writer.writeheader()

    for row in reader:
        isbn = row['ISBN']
        book_info = get_book_info_by_isbn(isbn)
        if book_info:
            categories = book_info['volumeInfo'].get('categories', 'N/A')
            small_thumbnail = book_info['volumeInfo'].get('imageLinks', {}).get('smallThumbnail', 'N/A')
        else:
            categories = 'N/A'
            small_thumbnail = 'N/A'
        
        # Add the categories and small thumbnail to the current row
        row['Categories'] = ', '.join(categories) if isinstance(categories, list) else categories
        row['SmallThumbnail'] = small_thumbnail
        
        
        # Write the updated row to the output CSV
        writer.writerow(row)

print(f"Processed CSV and saved to {output_csv_path}")



In [ ]:
# Open the processed CSV and filter out any books without Category or thumbnail

import csv
# File paths for input and output CSVs
input_csv_path = 'another_updated_goodreads_library_export.csv'  # This is the CSV that already has "Categories" and "SmallThumbnail" columns
filtered_csv_path = 'filtered_books.csv'

# Read the input CSV and filter rows
with open(input_csv_path, mode='r', newline='') as infile, open(filtered_csv_path, mode='w', newline='') as outfile:
    reader = csv.DictReader(infile)
    writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)

    # Write the header to the filtered output CSV
    writer.writeheader()

    for row in reader:
        # Check if both "SmallThumbnail" and "Categories" are not 'N/A'
        if row['SmallThumbnail'] != 'N/A' and row['Categories'] != 'N/A':
            # Write the row to the filtered output CSV
            writer.writerow(row)

print(f"Filtered CSV and saved to {filtered_csv_path}")